In [14]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from scipy.stats import skew
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
import re
import pymorphy2
from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_squared_error, make_scorer

# Обычные преобразования

In [4]:
def combinations_two_features(df, couple_columns):
    for couple in couple_columns:
        df[couple[0] + "*" + couple[1]] = df[couple[0]] * df[couple[1]]
    return df

In [5]:
def polynomials(df, list_columns, degree):
    for col in list_columns:
        df[col + f"_s{degree}"] = df[col] * degree
    return df

In [7]:
def log_skew_columns(data, col_list, skew_coef):
    skewed_feats = data[col_list].apply(lambda x: skew(x.dropna())) #compute skewness
    skewed_feats = skewed_feats[skewed_feats > skew_coef]
    skewed_feats = skewed_feats.index
    data[skewed_feats] = np.log1p(data[skewed_feats])
    return data

# Количество пустых полей

In [13]:
def count_null_fields_tech(data,  cols, name_null_count = "null_count"):
    data[name_null_count] = [0] * len(data)
    
    for col in cols:
        data[name_null_count] =data[name_null_count] + data[col].apply(lambda x:  1 if pd.isna(x) else 0)  
    return data

# Количествое заявок

In [ ]:
rec_df_train = train.id_client.value_counts().reset_index()
rec_df_test = test.id_client.value_counts().reset_index()


def is_rec_num_client_fit_transform(data, id_field = "id_client"):
    def rec_(x,rec_df):
        try:
            if rec_df[rec_df["index"]== int(x)].iloc[0,1] == 1:
                return 0
            elif rec_df[rec_df["index"]== int(x)].iloc[0,1] > 1:
                return rec_df[rec_df["index"]== int(x)].iloc[0,1]
        except:
            return 0
    rec_df = data[id_field].value_counts().reset_index()
    data["num_appli"] = data[id_field].apply(lambda x: rec_(int(x), rec_df))
    return data

def is_rec_num_client_transform(data, rec_df, id_field = "id_client"):
    def rec_(x,rec_df):
        try:
            if rec_df[rec_df["index"]== int(x)].iloc[0,1] == 1:
                return 0
            elif rec_df[rec_df["index"]== int(x)].iloc[0,1] > 1:
                return rec_df[rec_df["index"]== int(x)].iloc[0,1]
        except:
            return 0
    
    data["num_appli"] = data[id_field].apply(lambda x: rec_(int(x), rec_df))
    return data

def smart_concat(rec_df_train, rec_df_test):
    def add_rec_df_train(x, list_rec, rec_df_train, rec_df_test ):
        if x in list_rec:
            return rec_df_train[rec_df_train["index"]== x].iloc[0,1] + rec_df_test[rec_df_test["index"]== x].iloc[0,1]
        else:
            return  rec_df_train[rec_df_train["index"]== x].iloc[0,1] 
    list_rec_test = rec_df_test["index"].to_list()
    list_rec_train = rec_df_train["index"].to_list()
    
    rec_df_test["id_client"] = rec_df_test["index"].apply(lambda x: add_rec_df_train(x,list_rec_train, rec_df_test, rec_df_train))
    
    return rec_df_test

# Повторный ли клиент

In [ ]:
def is_rec_client_fit_transform(data, id_field = "id_client"):
    def rec_(x,rec_df):
        try:
            if rec_df[rec_df["index"]== int(x)].iloc[0,1] == 1:
                return 0
            elif rec_df[rec_df["index"]== int(x)].iloc[0,1] > 1:
                return 1
        except:
            return 0
    rec_df = data[id_field].value_counts().reset_index()
    data["is_rec"] = data[id_field].apply(lambda x: rec_(int(x), rec_df))
    return data

def is_rec_client_transform(data, rec_df, id_field = "id_client"):
    def rec_(x,rec_df):
        try:
            if rec_df[rec_df["index"]== int(x)].iloc[0,1] == 1:
                return 0
            elif rec_df[rec_df["index"]== int(x)].iloc[0,1] > 1:
                return 1
        except:
            return 0
    
    data["is_rec"] = data[id_field].apply(lambda x: rec_(int(x), rec_df))
    return data